# Weather and Mood-based Spotify Playlist Generator
This notebook implements a simple application that generates Spotify playlist recommendations based on weather conditions and user mood.

In [ ]:
# Import necessary libraries
import requests
import json
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [ ]:
# Configuration for Weather API (OpenWeatherMap)
WEATHER_API_KEY = "your_openweathermap_api_key"
WEATHER_BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

def get_weather(city):
    """
    Fetch current weather data for a given city

    Args:
        city (str): City name

    Returns:
        dict: Weather data
    """
    params = {
        'q': city,
        'appid': WEATHER_API_KEY,
        'units': 'metric'
    }

    response = requests.get(WEATHER_BASE_URL, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching weather data: {response.status_code}")
        return None

In [ ]:
# Configuration for Spotify API
SPOTIFY_CLIENT_ID = "your_spotify_client_id"
SPOTIFY_CLIENT_SECRET = "your_spotify_client_secret"
SPOTIFY_AUTH_URL = "https://accounts.spotify.com/api/token"
SPOTIFY_BASE_URL = "https://api.spotify.com/v1"

def get_spotify_token():
    """
    Get access token for Spotify API

    Returns:
        str: Access token
    """
    auth_response = requests.post(SPOTIFY_AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': SPOTIFY_CLIENT_ID,
        'client_secret': SPOTIFY_CLIENT_SECRET,
    })

    auth_data = auth_response.json()
    return auth_data['access_token']

def get_playlist_recommendations(weather_condition, mood):
    """
    Get playlist recommendations based on weather and mood

    Args:
        weather_condition (str): Current weather condition
        mood (str): User's current mood

    Returns:
        list: Recommended playlists
    """
    token = get_spotify_token()
    headers = {
        'Authorization': f'Bearer {token}'
    }

    # Map weather conditions to genres
    weather_genre_map = {
        'Clear': ['pop', 'dance', 'happy'],
        'Clouds': ['indie', 'alternative', 'chill'],
        'Rain': ['acoustic', 'jazz', 'rainy-day'],
        'Snow': ['classical', 'ambient', 'winter'],
        'Thunderstorm': ['rock', 'epic', 'intense'],
        'Mist': ['lo-fi', 'study', 'focus']
    }

    # Map moods to descriptive terms for search
    mood_map = {
        'happy': ['feel-good', 'upbeat', 'happy'],
        'sad': ['melancholy', 'sad', 'chill'],
        'energetic': ['workout', 'dance', 'energy'],
        'relaxed': ['chill', 'ambient', 'relaxing'],
        'focused': ['focus', 'concentration', 'study']
    }

    # Get genres based on weather (default to pop if weather condition not found)
    genres = weather_genre_map.get(weather_condition, ['pop'])

    # Get mood descriptors
    mood_terms = mood_map.get(mood.lower(), ['feel-good'])

    # Combine weather and mood to create search query
    search_term = f"{genres[0]} {mood_terms[0]} playlist"

    search_url = f"{SPOTIFY_BASE_URL}/search"
    params = {
        'q': search_term,
        'type': 'playlist',
        'limit': 5
    }

    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200:
        playlists = response.json()['playlists']['items']
        return playlists
    else:
        print(f"Error fetching playlists: {response.status_code}")
        return []

# Weather-Mood Playlist Generator Interface

Below is the interface for generating Spotify playlist recommendations based on your current weather and mood.

In [ ]:
# Simple user interface using IPython widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

def show_recommendations